In [2]:
import mysql.connector
import pandas as pd
import unicodedata


---

<h3><center>Funciones</center></h3>

In [6]:
def conectar_mysql():
    try:
        conexion = mysql.connector.connect(
            host='localhost',
            user='root',
            password='1533542415',
            database='quantyle_analitics'
        )
        print("¡Conexión establecida correctamente!")
        return conexion
    except mysql.connector.Error as error:
        print("Error al conectarse a MySQL:", error)
        return None
    

---

In [4]:
# Establecer la conexión con la base de datos
conexion = conectar_mysql()

# Verificar si la conexión se realizó con éxito
if conexion.is_connected():
    print("Conexión exitosa")

# Realizar operaciones en la base de datos, por ejemplo, ejecutar una consulta SQL
cursor = conexion.cursor()

# Ejemplo de consulta SQL
consulta = "SELECT * FROM categories_yelp"

# Ejecutar la consulta
cursor.execute(consulta)

# Obtener los resultados
resultados = cursor.fetchall()

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()


¡Conexión establecida correctamente!
Conexión exitosa


# 1) State

In [5]:
state = pd.read_parquet(r'..\datasets\processed\bd\1_states.parquet.gz')

conexion = conectar_mysql()

# Insertar filas en la tabla state
cursor = conexion.cursor()
for index, row in state.iterrows():
    cursor.execute("INSERT INTO state (state) VALUES (%s)", (row['state'],))  # Asegúrate de tener la coma al final

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 2) Categories

In [6]:
categories = pd.read_parquet(r'..\datasets\processed\bd\2_categories.parquet.gz')

# Aplicar strip() a la columna 'name' para eliminar espacios en blanco adicionales
categories['name'] = categories['name'].str.strip()

# Convertir la columna 'name' a minúsculas para uniformizar los datos
categories['name'] = categories['name'].str.lower()

# Eliminar las filas duplicadas basadas en la columna 'name'
categories.drop_duplicates(subset=['name'], keep='first', inplace=True)

conexion = conectar_mysql()

# Insertar filas en la tabla categories
cursor = conexion.cursor()
for index, row in categories.iterrows():
    cursor.execute("INSERT INTO categories (categories_id , name) VALUES (%s, %s)", (row['categories_id'], row['name'],))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 3) User yelp

In [7]:
user_yelp = pd.read_parquet(r'..\datasets\processed\bd\3_user_yelp.parquet.gz')
user_yelp['creation'] = user_yelp['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_yelp.iterrows():
    cursor.execute("INSERT INTO user_yelp (user_id, name, creation, review_count, influence, stars) VALUES (%s, %s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['influence'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()



¡Conexión establecida correctamente!


KeyboardInterrupt: 

# 4) User google

In [ ]:
user_google = pd.read_parquet(r'..\datasets\processed\bd\4_user_google.parquet.gz')
user_google['creation'] = user_google['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')
user_google.drop_duplicates(inplace=True)
user_google['user_id'] = user_google['user_id'].astype(str)
user_google['name'] = user_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
user_google.drop(2023700,inplace=True)
user_google.dropna(inplace=True)
user_google = user_google.groupby(['user_id','name','creation','review_count'])['stars'].mean().reset_index()
user_google = user_google.drop_duplicates(subset=['user_id'], keep='first')


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in user_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    user_google[column] = user_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_google.iterrows():
    cursor.execute("INSERT INTO user_google (user_id, name, creation, review_count, stars) VALUES (%s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


KeyboardInterrupt: 

# 5) Bussiness google

In [ ]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')
business_google = pd.read_parquet(r'..\datasets\processed\bd\5_bussiness_google.parquet.gz')


business_google.rename(columns={'state': 'state_id'}, inplace=True)
business_google["state_id"] = business_google["state_id"].replace("Florida", 1)
business_google["state_id"] = business_google["state_id"].replace("California", 2)
business_google["state_id"] = business_google["state_id"].replace("Illinois", 3)
business_google["state_id"] = business_google["state_id"].replace("New Jersey", 4)
business_google['name'] = business_google['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_google.dropna(inplace=True)
business_google = business_google.drop_duplicates(subset=['gmap_id'], keep='first')
business_google.drop(22003,inplace=True)
business_google['avg_stars'] = business_google['avg_stars'].round(2)

business_google = business_google[business_google['gmap_id'].isin(categories_google['gmap_id'].unique())]


def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_google[column] = business_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_google[['gmap_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_google (gmap_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 6) Business yelp

In [ ]:
business_yelp = pd.read_parquet(r'..\datasets\processed\bd\6_bussiness_yelp.parquet.gz')

business_yelp.rename(columns={'state': 'state_id'}, inplace=True)
business_yelp.rename(columns={'stars': 'avg_stars'}, inplace=True)
business_yelp["state_id"] = business_yelp["state_id"].replace("Florida", 1)
business_yelp["state_id"] = business_yelp["state_id"].replace("California", 2)
business_yelp["state_id"] = business_yelp["state_id"].replace("Illinois", 3)
business_yelp["state_id"] = business_yelp["state_id"].replace("New Jersey", 4)
business_yelp['name'] = business_yelp['name'].replace(r'[^a-zA-Z\s]', '', regex=True)
business_yelp.dropna(inplace=True)
business_yelp = business_yelp.drop_duplicates(subset=['business_id'], keep='first')
business_yelp['avg_stars'] = business_yelp['avg_stars'].round(2)

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in business_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    business_yelp[column] = business_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in business_yelp[['business_id', 'name', 'latitude', 'longitude', 'avg_stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_yelp (business_id, name, latitude, longitude, avg_stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 7) Categories google

In [ ]:
categories_google = pd.read_parquet(r'..\datasets\processed\bd\7_categories_google.parquet.gz')

categories_google = categories_google[categories_google['categories_id'].isin(categories['categories_id'])]

categories_google = categories_google[categories_google['gmap_id'].isin(business_google['gmap_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_google.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_google[column] = categories_google[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)
    
conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_google[['gmap_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_google (gmap_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 8) Categories yelp

In [ ]:
categories_yelp = pd.read_parquet(r'..\datasets\processed\bd\8_categories_yelp.parquet.gz')

categories_yelp = categories_yelp[categories_yelp['categories_id'].isin(categories['categories_id'])]

categories_yelp = categories_yelp[categories_yelp['business_id'].isin(business_yelp['business_id'])]

def corregir_codificacion(texto):
    try:
        return texto.encode('latin-1').decode('utf-8', 'ignore')
    except Exception:
        return unicodedata.normalize('NFKD', texto).encode('latin-1', 'ignore').decode('utf-8', 'ignore')


# Recorre todas las columnas del DataFrame
for column in categories_yelp.columns:
    # Intenta corregir los caracteres no codificados como UTF-8 en cada celda
    categories_yelp[column] = categories_yelp[column].apply(lambda x: corregir_codificacion(str(x)) if isinstance(x, str) else x)

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in categories_yelp[['business_id', 'categories_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO categories_yelp (business_id, categories_id) VALUES (%s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


¡Conexión establecida correctamente!


# 9) Reviews google

In [10]:
reviews_google = pd.read_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz')

reviews_google['date'] = reviews_google['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
reviews_google['resp_date'] = reviews_google['resp_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

conexion = conectar_mysql()

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in reviews_google[['user_id', 'gmap_id','stars','sentiment','date','sentiment','resp_sentiment','resp_date']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO reviews_google (user_id, gmap_id, stars, sentiment, date, sentiment, resp_sentiment, resp_date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()

¡Conexión establecida correctamente!


OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 8 EOF occurred in violation of protocol (_ssl.c:2427)

In [9]:
reviews_google = pd.read_parquet(r'..\datasets\processed\bd\9_reviews_google.parquet.gz')

reviews_google['date'] = reviews_google['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
reviews_google['resp_date'] = reviews_google['resp_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
reviews_google

,user_id,gmap_id,stars,sentiment,date,resp_sentiment,resp_date
0,1.089912e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-01-06 05:12:07,0.0,NaN
1,1.112903e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2021-02-09 05:47:28,0.0,NaN
2,1.126404e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,4,0,2020-03-08 05:04:42,0.0,NaN
3,1.174403e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2019-03-07 05:56:56,0.0,NaN
4,1.005808e+20,0x80c2c778e3b73d33:0xbdc58662a4a97d49,5,2,2017-05-16 05:01:41,0.0,NaN
...,...,...,...,...,...,...,...
2452767,1.091054e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2016-09-01 01:03:33,0.0,NaN
2452768,1.159805e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2017-07-26 19:09:52,0.0,NaN
2452769,1.159471e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,4,0,2015-11-16 17:06:38,0.0,NaN
2452770,1.179236e+20,0x880fce33edd16c7b:0x44e066027b82ad8d,3,0,2016-10-06 10:41:33,0.0,NaN
